In [1]:
import numpy as np
import pandas as pd

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth

In [3]:
client_id = '8bbd5f57c57e4d159a9666bd4b23d26a'
client_secret = '816dc0d82ec24f4c8cfa34373b89e067'
redirect_uri = 'http://localhost:7777/callback'

In [4]:
kaggle_spotify = pd.read_csv('spotify_data.csv')
kaggle_spotify.head(5)

,Unnamed: 0,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0,Jason Mraz,I Won't Give Up,53QF56cjZA9RTuuMZDrSA6,68,2012,acoustic,0.483,0.303,4,-10.058,1,0.0429,0.6940,0.000000,0.1150,0.139,133.406,240166,3
1,1,Jason Mraz,93 Million Miles,1s8tP3jP4GZcyHDsjvw218,50,2012,acoustic,0.572,0.454,3,-10.286,1,0.0258,0.4770,0.000014,0.0974,0.515,140.182,216387,4
2,2,Joshua Hyslop,Do Not Let Me Go,7BRCa8MPiyuvr2VU3O9W0F,57,2012,acoustic,0.409,0.234,3,-13.711,1,0.0323,0.3380,0.000050,0.0895,0.145,139.832,158960,4
3,3,Boyce Avenue,Fast Car,63wsZUhUZLlh1OsyrZq7sz,58,2012,acoustic,0.392,0.251,10,-9.845,1,0.0363,0.8070,0.000000,0.0797,0.508,204.961,304293,4
4,4,Andrew Belle,Sky's Still Blue,6nXIYClvJAfi6ujLiKqEq8,54,2012,acoustic,0.430,0.791,6,-5.419,0,0.0302,0.0726,0.019300,0.1100,0.217,171.864,244320,4


In [5]:
kaggle_spotify.shape

(1159764, 20)

In [6]:
kaggle_spotify_variables = kaggle_spotify.columns.tolist()
kaggle_spotify_variables

['Unnamed: 0',
 'artist_name',
 'track_name',
 'track_id',
 'popularity',
 'year',
 'genre',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'duration_ms',
 'time_signature']

In [7]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                               client_secret=client_secret,
                                               redirect_uri=redirect_uri,
                                               scope='user-library-read'))

In [24]:
# Create an empty list to store track details
tracks_data = []

# Initialize variables for pagination
offset = 0
limit = 50  # Number of tracks per request

while True:
    results = sp.current_user_saved_tracks(offset=offset, limit=limit)
    tracks = [item['track'] for item in results['items']]

    if not tracks:
        break

    for track in tracks:
        track_id = track['id']
        track_name = track['name']
        artist_name = track['artists'][0]['name']
        tracks_data.append({'Source': 'Library',
                            'Artist': artist_name,
                            'Track Name': track_name,
                            'Track ID': track_id})

    offset += limit

# Define your Spotify username and playlist names
username = '21kk3hvbn7qcsjc6r6asdbnqi'
playlist_names = ['Freakin Lentils 🍛', 'September ✈️ 🌄', 'Chicken Patty 🥧', 'Disconuts 🍩', 'chips and gravy 🍟'
                 , 'Playlist #29', 'Good.Year 🥡', 'Broodles 🍜', 'Pastry Flakes 🥐', 'lllamb Sauce', 'Groovy Taco 🌮'
                 , 'Shikanjabeen 🍹', 'PINEAPPLE 🍍', 'Hot Chocolate 🥛']

# Function to get all tracks from a playlist
def get_playlist_tracks(username, playlist_name):
    playlists = sp.user_playlists(username)
    for playlist in playlists['items']:
        if playlist['name'] == playlist_name:
            results = sp.playlist_tracks(playlist['id'])
            tracks = results['items']
            while results['next']:
                results = sp.next(results)
                tracks.extend(results['items'])
            return tracks
    return None

# Get all tracks from the specified playlists
for playlist_name in playlist_names:
    tracks = get_playlist_tracks(username, playlist_name)
    if tracks:
        for track in tracks:
            track_info = track['track']
            artist = track_info['artists'][0]['name']
            track_name = track_info['name']
            track_id = track_info['id']
            tracks_data.append({'Source': 'Playlist',
                                'Artist': artist,
                                'Track Name': track_name,
                                'Track ID': track_id})

# Create a DataFrame
my_tracks_df = pd.DataFrame(tracks_data)

# Remove duplicate tracks based on Track ID
my_tracks_df = my_tracks_df.drop_duplicates(subset=['Track ID'])

# Reset the index
my_tracks_df.reset_index(drop=True, inplace=True)

# Print the resulting DataFrame
my_tracks_df.head(5)

,Source,Artist,Track Name,Track ID
0,Library,Jacob Collier,Wherever I Go (feat. Lawrence & Michael McDonald),77PlrIDSmmWwCj7vgNQiR4
1,Library,Gavin DeGraw,I Don't Want to Be,17yReVjsDb7Hlvjqz6jhDm
2,Library,Jai Paul,Jasmine - Demo,44LAHDPaM5dvkmmD5w1W84
3,Library,Peggy Gou,(It Goes Like) Nanana - Edit,23RoR84KodL5HWvUTneQ1w
4,Library,Laura Mvula,Sing to the Moon,7FP5y5nA6F7eIjc3cPGQiz


In [25]:
my_tracks_df.shape

(7055, 4)

In [30]:
import time

# Use the previously created my_tracks_df DataFrame

# Get the track IDs
track_ids = my_tracks_df['Track ID'].tolist()

# Initialize an empty list to store audio features
audio_features_data = []

# Initialize variables for pagination
limit = 50  # Number of tracks per request

for i in range(0, len(track_ids), limit):
    track_subset = track_ids[i:i+limit]
    retries = 0
    
    while retries < 3:  # You can adjust the number of retries as needed
        try:
            audio_features = sp.audio_features(tracks=track_subset)
            if audio_features is not None:
                audio_features_data.extend(audio_features)
            break
        except Exception as e:
            print(f"An error occurred: {e}. Retrying after 10 seconds...")
            time.sleep(10)  # Wait for 10 seconds before retrying
            retries += 1

# Create a DataFrame for audio features
audio_features_df = pd.DataFrame(audio_features_data)

# Print the resulting DataFrame
audio_features_df.head(5)

Max Retries reached


An error occurred: http status: 429, code:-1 - /v1/audio-features/?ids=77PlrIDSmmWwCj7vgNQiR4,17yReVjsDb7Hlvjqz6jhDm,44LAHDPaM5dvkmmD5w1W84,23RoR84KodL5HWvUTneQ1w,7FP5y5nA6F7eIjc3cPGQiz,7xSWHCMlpRJHCGakzH8TYc,1P9EVGR0fFRVaLL8edTlGR,0sY6ZUTh4yoctD8VIXz339,2GiJYvgVaD2HtM8GqD9EgQ,2sgCXHg97idWBFoXV7M5uK,5IRbY7UwwTfJB0M3MGYLml,4s3xXkjGYSwI01zTp5f1by,2aJPnC4744fGWzV66VBO8h,2PLi7OmleXPNBrGLon3sUy,1HQljdSKC7dUzYynbdRT04,3hxoYmSHzIBwSS2bPTOfbg,0PNU96DGl5PRswbstQgbtv,6Nle9hKrkL1wQpwNfEkxjh,1rL0CJe38sZ9fQCe6n4xUq,01aWeWZLOuDugDTHEyySzD,0Dv2dILPDIqPjCqGbW9WTN,7rUjaX9U6pquRTgqDlGnfI,6e65fH1wClamA2QPI5HUi3,4HCcvFdHfwR2u3WPPPVRv6,59tcQJiHwApzE4h6yVnL9i,4FKuIlfFCIcvpg7M2HJhwx,7aG0pmSfkeWgMEtlkXHLsV,1vbe9sh7U9vdMceAhitWr2,1oQLzryECAAeX7GmddL6jn,0hQNwOFER7LS4Bzbk4jPov,18QgJOKiWqWu5BQgTNfbq0,1Hawc2sA1Ty2DVlWg5NwrB,3vDedfcgwh5diQzCh8D3dG,4UYX7vKkU7QC6G62eKlGgc,57Oj7YHoMrwVbR88jJuDL8,5TIs921w5iOz6uRkel8AWC,6sjaBYmJVG7Xw82YHkDqRv,7HfAdACcnJnnbwqY6IBW58,7qhiD7dVX37al4Ytw9vplh,38E7U60ipLSTVDcPIuVzZw,7wEGeUzpP

Max Retries reached


An error occurred: http status: 429, code:-1 - /v1/audio-features/?ids=77PlrIDSmmWwCj7vgNQiR4,17yReVjsDb7Hlvjqz6jhDm,44LAHDPaM5dvkmmD5w1W84,23RoR84KodL5HWvUTneQ1w,7FP5y5nA6F7eIjc3cPGQiz,7xSWHCMlpRJHCGakzH8TYc,1P9EVGR0fFRVaLL8edTlGR,0sY6ZUTh4yoctD8VIXz339,2GiJYvgVaD2HtM8GqD9EgQ,2sgCXHg97idWBFoXV7M5uK,5IRbY7UwwTfJB0M3MGYLml,4s3xXkjGYSwI01zTp5f1by,2aJPnC4744fGWzV66VBO8h,2PLi7OmleXPNBrGLon3sUy,1HQljdSKC7dUzYynbdRT04,3hxoYmSHzIBwSS2bPTOfbg,0PNU96DGl5PRswbstQgbtv,6Nle9hKrkL1wQpwNfEkxjh,1rL0CJe38sZ9fQCe6n4xUq,01aWeWZLOuDugDTHEyySzD,0Dv2dILPDIqPjCqGbW9WTN,7rUjaX9U6pquRTgqDlGnfI,6e65fH1wClamA2QPI5HUi3,4HCcvFdHfwR2u3WPPPVRv6,59tcQJiHwApzE4h6yVnL9i,4FKuIlfFCIcvpg7M2HJhwx,7aG0pmSfkeWgMEtlkXHLsV,1vbe9sh7U9vdMceAhitWr2,1oQLzryECAAeX7GmddL6jn,0hQNwOFER7LS4Bzbk4jPov,18QgJOKiWqWu5BQgTNfbq0,1Hawc2sA1Ty2DVlWg5NwrB,3vDedfcgwh5diQzCh8D3dG,4UYX7vKkU7QC6G62eKlGgc,57Oj7YHoMrwVbR88jJuDL8,5TIs921w5iOz6uRkel8AWC,6sjaBYmJVG7Xw82YHkDqRv,7HfAdACcnJnnbwqY6IBW58,7qhiD7dVX37al4Ytw9vplh,38E7U60ipLSTVDcPIuVzZw,7wEGeUzpP

Max Retries reached


...

Max Retries reached


An error occurred: http status: 429, code:-1 - /v1/audio-features/?ids=54xEV1IGBwpCi8h566crhV,5YgFl0b1J3uMq4bU7HuY0P,6ta5yavnnEfCE4faU0jebM,0ERBXd2jq2ae81zXlekbEy,4xWLo3R0jjTVOdmtjC5MKI,1KN2z32nlbkmh0I8XXlNWD,7D1YY4ofOKjX3wB9HkadLK,6TC0snyor5XOJzP6dGxRkn,0DzGpRoWQpwVAzy3Gn5EZF,36ChespiFdPjZ7xgcLG3dO,68j5DYmCikBzcp60Sv6xWO,6Rg1H1aIrg99gsOKCEEDjh,3bwAS8cN7Xtzvv5UurJOOX,7KoXzM3IboDJxRl7DXIT7y,0zyhmMx0b3CFDfyOuh2ZmC,1wq5gRZ5JRn25N4JCUg1oO,39WFZVKifA8Fl2TZI2kp0S,28wERzXJWMb4Qx1bf9EXyW,47w6S27JF4Zvrz1Wd6R66Z,1E6kgbqLP4GPd4gsKab6Wt,5Bq4G0xWjiDZDkx01IKMCc,4wwuuiyk9cwEF7MZGRafrz,6HadZejJaG1KfJkXC4HAbv,5Vx50cLdtXDZ8cR23CP2nM,1DZFqyULjH1GplYnWa5psW,4UZlEcQmkAeh4mYokk8TNs,1WSW4VT8o81I4XIHINIXvC,3b9MUQ7uiQHhU420B6f8Ta,3tsFWhe45D6CGFSMA77T3B,4C0w63kqpLDNfTGsbLp8VG,6C7WlMpQr4V8lCkkrGj0MK,6AOiHpFFwVY52p3EaA7eHl,0upn2c2d9jGu6nI6hiTjW9,3rJWuc53MmmGSMVGPZKynS,6OyjHEusfnpjMxewx4msTA,1XTOKwfpo91ehYbNwGaB6O,511InmWye3UrDWkcE4SmT4,3ma3vkzDmUYudCDhbSDq0l,5dU7R9IGlszJCfWlRZVbAf,2QglMIKYxa06BFSXG6KwvC,5U3i59kbT

Max Retries reached


An error occurred: http status: 429, code:-1 - /v1/audio-features/?ids=54xEV1IGBwpCi8h566crhV,5YgFl0b1J3uMq4bU7HuY0P,6ta5yavnnEfCE4faU0jebM,0ERBXd2jq2ae81zXlekbEy,4xWLo3R0jjTVOdmtjC5MKI,1KN2z32nlbkmh0I8XXlNWD,7D1YY4ofOKjX3wB9HkadLK,6TC0snyor5XOJzP6dGxRkn,0DzGpRoWQpwVAzy3Gn5EZF,36ChespiFdPjZ7xgcLG3dO,68j5DYmCikBzcp60Sv6xWO,6Rg1H1aIrg99gsOKCEEDjh,3bwAS8cN7Xtzvv5UurJOOX,7KoXzM3IboDJxRl7DXIT7y,0zyhmMx0b3CFDfyOuh2ZmC,1wq5gRZ5JRn25N4JCUg1oO,39WFZVKifA8Fl2TZI2kp0S,28wERzXJWMb4Qx1bf9EXyW,47w6S27JF4Zvrz1Wd6R66Z,1E6kgbqLP4GPd4gsKab6Wt,5Bq4G0xWjiDZDkx01IKMCc,4wwuuiyk9cwEF7MZGRafrz,6HadZejJaG1KfJkXC4HAbv,5Vx50cLdtXDZ8cR23CP2nM,1DZFqyULjH1GplYnWa5psW,4UZlEcQmkAeh4mYokk8TNs,1WSW4VT8o81I4XIHINIXvC,3b9MUQ7uiQHhU420B6f8Ta,3tsFWhe45D6CGFSMA77T3B,4C0w63kqpLDNfTGsbLp8VG,6C7WlMpQr4V8lCkkrGj0MK,6AOiHpFFwVY52p3EaA7eHl,0upn2c2d9jGu6nI6hiTjW9,3rJWuc53MmmGSMVGPZKynS,6OyjHEusfnpjMxewx4msTA,1XTOKwfpo91ehYbNwGaB6O,511InmWye3UrDWkcE4SmT4,3ma3vkzDmUYudCDhbSDq0l,5dU7R9IGlszJCfWlRZVbAf,2QglMIKYxa06BFSXG6KwvC,5U3i59kbT

KeyboardInterrupt: 

In [32]:
kaggle_spotify = kaggle_spotify.rename(columns={'track_id': 'Track ID'})

In [55]:
merged_df = pd.merge(kaggle_spotify, my_tracks_df, on='Track ID', how='left')
merged_df.head(5)

,Unnamed: 0,artist_name,track_name,Track ID,popularity,year,genre,danceability,energy,key,...,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,Source,Artist,Track Name
0,0,Jason Mraz,I Won't Give Up,53QF56cjZA9RTuuMZDrSA6,68,2012,acoustic,0.483,0.303,4,...,0.6940,0.000000,0.1150,0.139,133.406,240166,3,NaN,NaN,NaN
1,1,Jason Mraz,93 Million Miles,1s8tP3jP4GZcyHDsjvw218,50,2012,acoustic,0.572,0.454,3,...,0.4770,0.000014,0.0974,0.515,140.182,216387,4,NaN,NaN,NaN
2,2,Joshua Hyslop,Do Not Let Me Go,7BRCa8MPiyuvr2VU3O9W0F,57,2012,acoustic,0.409,0.234,3,...,0.3380,0.000050,0.0895,0.145,139.832,158960,4,NaN,NaN,NaN
3,3,Boyce Avenue,Fast Car,63wsZUhUZLlh1OsyrZq7sz,58,2012,acoustic,0.392,0.251,10,...,0.8070,0.000000,0.0797,0.508,204.961,304293,4,NaN,NaN,NaN
4,4,Andrew Belle,Sky's Still Blue,6nXIYClvJAfi6ujLiKqEq8,54,2012,acoustic,0.430,0.791,6,...,0.0726,0.019300,0.1100,0.217,171.864,244320,4,NaN,NaN,NaN


In [56]:
merged_df.shape

(1159764, 23)

In [57]:
filtered_df = merged_df[merged_df['Source'].notna()]

filtered_df.shape

(1324, 23)

In [58]:
merged_df.to_csv('merged_df.csv', index=False)